This notebook showcases the usage of the `MBLlearning`-package.
It loads all necessary data and the pretrained model(s) but can also incorporate training the model from scratch.
All follow-up code is used to recreate every plot from the result section in the main text of the arXiv-paper

For additional plots in the appendix (especially the toy model), refer to the `toy-example`-notebook

In [ ]:
# Load relevant packages, in particular, the MBLlearning-package
# load pytorch for training
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# load finite-size scaling analysis package
import fssa
# standard libraries for data processing and plotting
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import axes3d # for 3d plots
import matplotlib as mpl
%matplotlib inline

In [ ]:
from MBLlearning.learn_inds.approximation import Approximator
from MBLlearning.learn_inds import recurrent
import MBLlearning.utils as utils
import MBLlearning.utils.plotting as plot

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from MBLlearning.global_config import get_config, diction
DTYPE, DEVICE = get_config()
NUM_DEVICES = torch.cuda.device_count() if torch.cuda.is_available() else 1
print("Using {} device(s) of type {}".format(NUM_DEVICES,DEVICE))

Training of the RNN-set-up
---

In [ ]:
# all relevant hyperparameters and data setting go here

################################################################################################################
# data settings
fileloc = "data/indicators_train_Uniform_L_{}_eps_{}.txt"
L = [10,12,14]
L_small = [10,12] # for quantitative extrapolation testing
L_extra = [16,18] # for qualitative  extrapolation testing
eps = np.arange(1,20)/20
single_inds = [0,3,5]
######## transfer learning indicators #######
init_inds = [0,1]
transfer_inds = [2]
#############################################
sorting = False
N_train = 1000
################################################################################################################
# hyperparameters of the model
feature_size = 8 # single number for RNN feature extractor
depth = 1 # depth of RNN-cell, see documentation of nn.GRU or nn.LSTM
use_LSTM = False # switch between GRU or LSTM cell
hidden_size = [10] # array of hidden dimensions for the hidden layer for fully-connected NN or None-Type
seq_division = 2 # preprocessing tuple size e.g. [h1,h2,h3] --> [(h1,h2),(h2,h3),(h3,h1)]
# training procedure
optimizer = optim.Adam
opt_params = (1e-4,) # learning rate, other parameters left as default
N_epochs = 20
batch_size = 32 # batch size per device
model_name_scheme = "trained_models/large_model_{}_hidden_size_{}_{}.pt" # {full, small, switched} x {rnn, model}
                                                                         # to be filled in
retrain_model = False # checks for availability of existing model file first.
                      # Can be suppressed by setting this to True
retrain_N_trains = False # setting this to True reruns the generation of data for plot 4 in the paper
                         # it is very time-consuming though
N_trains_retrain = np.append(np.arange(1,11),[10,100,1000])
retrain_filename = "mse_N_train/{}_loss_L_{}_N_{}.txt"
N_trains_plot = np.array([1,2,4,10,100,1000])
mute_outputs = False  # verbose level during training
################################################################################################################
# save figures? Provide a folder for the images
save_figures = "plots_paper/" # set to None-Type for no saving of plots
################################################################################################################

In [ ]:

neural_net = Approximator(L,energies=eps)
# data loading and model init
neural_net.set_up(fileloc,
                  rnn_params=(feature_size,depth),
                  model_params=hidden_size,
                  seq_division=seq_division,
                  use_LSTM=use_LSTM,
                  learn_single_ind=single_inds,
                  N_train=N_train)
# resets the model parameters but can also be used for altering the model architecture
neural_net.reset_model()
# regardless of training, set the optimizer and its parameters
neural_net.optimizer = optimizer
neural_net.opt_params = opt_params

print()
print("Hidden size =",hidden_size[0])
model_name = model_name_scheme.format("full",hidden_size[0],"{}") # fill in first place-holder
neural_net.reset_model()
if retrain_model:
    print("Retraining model from scratch ...")
    neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
    print("Retraining finished, saving to file.")
    neural_net.save(model_name)
else:
    print("Trying to load model from file ...")
    try:
        neural_net.load(model_name)
    except:
        print("File not found, training the model ...")
        neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
        print("Training finished, saving to file.")
        neural_net.save(model_name)
    else:
        print("Model loaded from file.")

# print the number of parameters in the RNN and the consecutive model
neural_net.num_params()

# create plot 3 in paper
temp = None if save_figures is None else save_figures + "indicator_plot_{}"

plot.plot_inds(neural_net,label_idxs=single_inds,savename=temp)

print(neural_net.Lvals)
plot.plot_inds_extrapolation(neural_net,[10,12,14,16,18,20,30],eps,label_idxs=single_inds)
print(neural_net.Lvals)

# check whether to retrain data with reduced N_train but increased N_epochs
# by default, load from file
if retrain_N_trains:
    N_max = max(N_trains_retrain)
    num_reps = 2

    #mses = np.zeros((num_reps,len(N_trains),len(L)))
    losses = []

    for i,N in enumerate(N_trains_retrain):
        # set up network
        temp_losses = []
        temp_net = Approximator(L,energies=eps)
        temp_net.set_up(fileloc,
                        rnn_params=(feature_size,depth),
                        model_params=hidden_sizes,
                        seq_division=seq_division,
                        use_LSTM=use_LSTM,
                        learn_single_ind=single_inds,
                        N_train=N)

        temp_net.optimizer = optimizer
        temp_net.opt_params = opt_params

        for k in range(num_reps):            
            temp = np.infty
            while temp > 1:
                # retry if training does not succeed which is visible in a high loss
                temp_net.reset_model()
                temp_net.Lvals = L_small # train only on smaller data file
                temp_epochs = int(N_epochs*N_max/N)
                temp_N = len(temp_net.data[L[0]]["train"]["hcorr"])
                print("Training with {} data points for {} epochs".format(temp_N,temp_epochs))
                _, loss_curve = temp_net.train(temp_epochs,batch_size*NUM_DEVICES,mute_outputs=True,L_tests=L)
                temp_net.Lvals = L # test on all test data
                temp = np.mean(loss_curve[0,-1]) # retry if final loss is above threshold
                #for j,l in enumerate(L):
                #    h, singlescores, targets = temp_net.predict(l)
                #    mse = ((singlescores-targets)**2).mean()
                #    mses[k,i,j] = mse
                #    print("MSE on L={} on the test set was {}".format(l,mse))
                #    temp = mse # retry if too large
            temp_losses.append(loss_curve)
        #mse_avg = np.mean(mses,axis=0)
        #mse_std = np.std(mses,axis=0)/np.sqrt(num_reps)
        losses.append(temp_losses)
    
    # save all losses
    for j,l in enumerate(L):
        for k,key in enumerate(["train","test"]):
            for loss,N in zip(losses,N_trains):
                np.savetxt(retrain_filename.format(key,l,N),np.array(loss)[:,k,:,j].T)
                print(np.array(loss)[:,k,:,j].T.shape)

    
prefactor = len(neural_net.data[L[0]]["train"]["h"])*len(L_small)/batch_size/NUM_DEVICES
# create plot 4 in paper
temp = None if save_figures is None else save_figures + "N_trains_plot_{}"
plot.plot_N_train_losses(12,N_trains_plot,N_epochs,prefactor,retrain_filename,savename=temp)
plot.plot_N_train_losses([10,14],N_trains_plot,N_epochs,prefactor,retrain_filename,savename=temp)

if False:
    # scatter plot losses
    color = ["red","blue","green"]
    temp = len(L)*[None]
    for i,(data,c,l) in enumerate(zip(mses.T,color,L)):
        #mean,std = filtered_statistics(data,threshold=1000)
        for N,dots in zip(N_trains,data):
            temp[i] = plt.scatter(N*np.ones_like(dots),dots,c=c)
    plt.legend(temp,L)
    plt.xscale('log')
    plt.show()    

    # create plot 4 in paper
    linestyles = ["bx","kx",None]
    for l,avg,std,style in zip(L,mse_avg.T,mse_std.T,linestyles):
        if l==14:
            plt.semilogx(N_trains,avg,color="orange",marker="*",label="L={}*".format(l))
            plt.errorbar(N_trains,avg,yerr=std,fmt="none",ecolor="orange")
        else:
            plt.semilogx(N_trains,avg,style,label="L={}".format(l))
            plt.errorbar(N_trains,avg,yerr=std,fmt="none",ecolor=style[0])
    plt.legend(fontsize="x-large")
    plt.xticks(fontsize="x-large")
    plt.yticks(fontsize="x-large")
    plt.ylabel("Mean MSE",fontsize="x-large")
    plt.xlabel("$N_{train}$ per disorder parameter $h$",fontsize="x-large")
    if save_figures is not None:
        plt.savefig(save_figures+"mse_N_train.pdf",orientation="landscape",dpi=600,bbox_inches="tight")
    plt.show()
else:
    print("Skipped old code. Check for deletion.")

# checks whether to (re)train the model or to load from file on smaller training data
model_name = model_name_scheme.format("small",hidden_size[0],"{}") # fill in first place-holder
if retrain_model:
    neural_net.Lvals = L_small # for training purposes
    print("Retraining model from scratch ...")
    neural_net.reset_model()
    neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
    print("Retraining finished, saving to file.")
    neural_net.save(model_name)
    neural_net.Lvals = L # for full test set analysis
else:
    print("Trying to load model from file ...")
    try:
        neural_net.load(model_name)
    except:
        neural_net.Lvals = L_small # for training purposes
        print("File not found, training the model ...")
        neural_net.reset_model()
        neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
        print("Training finished, saving to file.")
        neural_net.save(model_name)
        neural_net.Lvals = L # for full test set analysis
    else:
        print("Model loaded from file.")

# coefficient of determination on smaller training set (plot 5 in paper)
temp = None if save_figures is None else save_figures + "coefficient_of_determination"
fit_params = plot.plot_r2(neural_net,label_idxs=single_inds,savename=temp)

# FSSA
neural_net.swap_datasets()
neural_net.reset_model()
# check whether to retrain or load from file
model_name = model_name_scheme.format("fssa",hidden_size[0],"{}") # fill in first place-holder
if retrain_model:
    neural_net.Lvals = L_small # for training purposes
    neural_net.reset_model()
    print("Retraining model from scratch ...")
    neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
    print("Retraining finished, saving to file.")
    neural_net.save(model_name)
    neural_net.Lvals = L # for full test set analysis
else:
    print("Trying to load model from file ...")
    try:
        neural_net.load(model_name)
    except:
        neural_net.Lvals = L_small # for training purposes
        print("File not found, training the model ...")
        neural_net.reset_model()
        neural_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs)
        print("Training finished, saving to file.")
        neural_net.save(model_name)
        neural_net.Lvals = L # for full test set analysis
    else:
        print("Model loaded from file.")

# create plot 6 in paper
temp = None if save_figures is None else save_figures + "fssa_consistency"
fit_params = plot.plot_vanilla_fssa(neural_net,label_idxs=single_inds,savename=temp)

# create plot 7 in paper
temp = None if save_figures is None else save_figures + "fssa_extrapolate"
fit_params = plot.plot_extrapolated_fssa(neural_net,label_idxs=single_inds,savename=temp)

# transfer learning. Train on two indicators first, then drop the post-processing model
# Finally, retrain on the third indicator
# Reset data mess from FSSA
transfer_net = Approximator(L,energies=eps)
# data loading and model init
transfer_net.set_up(fileloc,
                  rnn_params=(feature_size,depth),
                  model_params=hidden_size,
                  seq_division=seq_division,
                  use_LSTM=use_LSTM,
                  learn_single_ind=single_inds,
                  N_train=N_train)
transfer_net.reset_model()
transfer_net.optimizer = optimizer
transfer_net.opt_params = opt_params
# check whether to retrain or load from file for initial training
model_name = model_name_scheme.format("transfer_init",hidden_size[0],"{}") # fill in first place-holder
if retrain_model:
    transfer_net.Lvals = L_small # for training purposes
    print("Retraining model from scratch ...")
    transfer_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs,
                     single_inds=init_inds
                    )
    print("Retraining finished, saving to file.")
    transfer_net.save(model_name)
    transfer_net.Lvals = L # for full test set analysis
else:
    print("Trying to load model from file ...")
    N_inds = transfer_net.N_inds
    try:
        # juggle with the number of indicators for loading procedure
        transfer_net.N_inds = len(init_inds)
        transfer_net.reset_model()
        transfer_net.load(model_name)
    except:
        transfer_net.N_inds = N_inds
        transfer_net.Lvals = L_small # for training purposes
        print("File not found, training the model ...")
        transfer_net.train(epochs=N_epochs,
                         batch_size=batch_size*NUM_DEVICES,
                         mute_outputs=mute_outputs,
                         single_inds=init_inds
                        )
        print("Training finished, saving to file.")
        transfer_net.save(model_name)
        transfer_net.Lvals = L # for full test set analysis
    else:
        transfer_net.N_inds = N_inds
        print("Model loaded from file.")
# check whether to retrain or load from file for transfer training
model_name = model_name_scheme.format("transfer",hidden_size[0],"{}") # fill in first place-holder
if retrain_model:
    transfer_net.Lvals = L_small # for training purposes
    print("Retraining model from scratch ...")
    transfer_net.train_transfer(ind_idxs=transfer_inds,
                     epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs
                    )
    print("Retraining finished, saving to file.")
    transfer_net.save(model_name)
    transfer_net.Lvals = L # for full test set analysis
else:
    print("Trying to load model from file ...")
    N_inds = transfer_net.N_inds
    try:
        # juggle with the number of indicators for loading procedure
        transfer_net.N_inds = len(transfer_inds)
        transfer_net.reset_model()
        transfer_net.load(model_name)
    except:
        transfer_net.N_inds = N_inds
        transfer_net.Lvals = L_small # for training purposes
        print("File not found, training the model ...")
        transfer_net.train_transfer(ind_idxs=transfer_inds,
                         epochs=N_epochs,
                         batch_size=batch_size*NUM_DEVICES,
                         mute_outputs=mute_outputs
                        )
        print("Training finished, saving to file.")
        transfer_net.save(model_name)
        transfer_net.Lvals = L # for full test set analysis
    else:
        transfer_net.N_inds = N_inds
        print("Model loaded from file.")

# set-up adversial network that learns the transfer indicator from scratch
adversary_net = Approximator(L,energies=eps)
# data loading and model init
adversary_net.set_up(fileloc,
                  rnn_params=(feature_size,depth),
                  model_params=hidden_size,
                  seq_division=seq_division,
                  use_LSTM=use_LSTM,
                  learn_single_ind=[single_inds[t] for t in transfer_inds],
                  N_train=N_train)
adversary_net.reset_model()
adversary_net.optimizer = optimizer
adversary_net.opt_params = opt_params
# check whether to retrain or load from file for initial training
model_name = model_name_scheme.format("transfer_adversary",hidden_size[0],"{}") # fill in first place-holder
if retrain_model:
    adversary_net.Lvals = L_small # for training purposes
    print("Retraining model from scratch ...")
    adversary_net.train(epochs=N_epochs,
                     batch_size=batch_size*NUM_DEVICES,
                     mute_outputs=mute_outputs
                    )
    print("Retraining finished, saving to file.")
    adversary_net.save(model_name)
    adversary_net.Lvals = L # for full test set analysis
else:
    print("Trying to load model from file ...")
    try:
        adversary_net.load(model_name)
    except:
        adversary_net.Lvals = L_small # for training purposes
        print("File not found, training the model ...")
        adversary_net.train(epochs=N_epochs,
                         batch_size=batch_size*NUM_DEVICES,
                         mute_outputs=mute_outputs
                        )
        print("Training finished, saving to file.")
        adversary_net.save(model_name)
        adversary_net.Lvals = L # for full test set analysis
    else:
        print("Model loaded from file.")

# reswap data sets for comparison
neural_net.swap_datasets()
# create plot 8 in paper concerning transfer learning
temp = None if save_figures is None else save_figures + "transfer_learning"
plot.plot_r2_comparison(transfer_net,neural_net,adversary_net,transfer_inds,savename=temp)

# create plot 11 from appendix
temp = None if save_figures is None else save_figures + "l_dependence"
plot.plot_L_dependent_regression(neural_net,label_idxs=single_inds,savename=temp)
